In [3]:
!ls -al /tf/notebooks/datasets/emotion/multi-modal/part3

total 1568
drwxrwxr-x 403 1005 1000 20480 May 13 11:33 .
drwxr-xr-x   5 1005 1000   210 Jul  6 18:34 ..
drwxrwxr-x   3 1005 1000  4096 May  7 11:52 KETI_MULTIMODAL_0000001000
drwxrwxr-x   3 1005 1000  4096 May  7 11:52 KETI_MULTIMODAL_0000001001
drwxrwxr-x   3 1005 1000  4096 May  7 11:53 KETI_MULTIMODAL_0000001002
drwxrwxr-x   3 1005 1000  4096 May  7 11:54 KETI_MULTIMODAL_0000001003
drwxrwxr-x   3 1005 1000  4096 May  7 11:55 KETI_MULTIMODAL_0000001004
drwxrwxr-x   3 1005 1000  4096 May  7 11:57 KETI_MULTIMODAL_0000001007
drwxrwxr-x   3 1005 1000  4096 May  7 12:00 KETI_MULTIMODAL_0000001008
drwxrwxr-x   3 1005 1000  4096 May  7 12:01 KETI_MULTIMODAL_0000001009
drwxrwxr-x   3 1005 1000  4096 May  7 12:01 KETI_MULTIMODAL_0000001010
drwxrwxr-x   3 1005 1000  4096 May  7 12:04 KETI_MULTIMODAL_0000001011
drwxrwxr-x   3 1005 1000  4096 May  7 12:05 KETI_MULTIMODAL_0000001012
drwxrwxr-x   3 1005 1000  4096 May  7 12:05 KETI_MULTIMODAL_0000001013
drwxrwxr-x   3 1005 1000  4096 May  7 12:05 

In [19]:
%%bash

source /root/.local/share/jupyter/kernels/rt-face-detect/bin/activate

# pip install pycodestyle pycodestyle_magic
# pip install flake8
# pip install ray
pip install line-profiler

  Stored in directory: /root/.cache/pip/wheels/05/7d/9b/aafbe8d78dc2b2c644d2efd2f060ab3258143860142575193a
Successfully built line-profiler


In [20]:
%load_ext pycodestyle_magic
%load_ext line_profiler

### 공통 전처리 부분 (Frame 단위)

#### inputs:
* 태깅파일: ~_interpolation.json
* 이미지파일: ~KM_0000000000.jpg
* 감정종류: 8가지 ('happiness', 'afraid', 'neutral', 'surprise', 'sadness', 'contempt', 'anger', 'disgust')
    * 예시: {'happiness': 0, 'afraid': 0, 'neutral': 10, 'surprise': 0, 'sadness': 0, 'contempt': 0, 'anger': 0, 'disgust': 0} 일 경우 'neutral'이 선정

In [111]:
import os
import json
import io
import numpy as np
import operator
import collections
import base64
import pandas as pd
from glob import glob
from IPython.display import HTML
import ray
import matplotlib.pyplot as plt
import copy

import logging

import cv2
from utils.inference import load_detection_model
from utils.inference import detect_faces
from utils.inference import apply_offsets

emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_alt2.xml'

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# multi-modal 데이터셋 다운로드 위치 입력

# 입력 zip파일의 part 번호
part_num = '1'

# input 폴더
base_folder = '/tf/notebooks/datasets/emotion/multi-modal/part'+part_num+'/'

# output 파일경로
prep_trainset_path='../datasets/kor_multi_modal/prep_part'+part_num+'.csv'
prep_trainset_frontal_face_path='../datasets/kor_multi_modal/prep_part'+part_num+'_frontal_face.csv'
prep_trainset_mid_path='../datasets/kor_multi_modal/prep_part'+part_num+'_mid.csv'

%matplotlib inline

I0721 00:34:00.956542 139652608198400 pyplot.py:211] Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## preprocess_jj.py part1~5번까지 실행부분 이후 단계
    * (수행 작업은 태깅데이터에서 얼굴 디텍트 되는 이미지만 뽑은 후 중간단계 인풋 데이터 생성)
## prep_part{%d}_frontal_face.csv 파일 생성

In [ ]:
# 한 파일당 약 2~3시간 소요(이미지를 읽어서 얼굴 디텍트되는지 확인 여부하는 것에 대부분 시간소요)
# python preprocess_jj.py <file_number>
예시: !python preprocess_jj.py 1

In [113]:
!ls -alh ../datasets/kor_multi_modal

total 4.8G
drwxr-xr-x 3 root root 4.0K Jul 21 00:33 .
drwxr-xr-x 4 root root  104 Jun 30 09:50 ..
drwxr-xr-x 2 root root  12K Jun 30 09:50 part1
-rw-r--r-- 1 root root 1.9G Jul 13 13:54 prep_col_all_mdf.csv
-rw-r--r-- 1 root root  76M Jul 13 15:29 prep_col_all_mdf_sample.csv
-rw-r--r-- 1 root root 240M Jul  6 18:18 prep_part1.csv
-rw-r--r-- 1 root root  39M Jul 20 20:13 prep_part1_frontal_face.csv
-rw-r--r-- 1 root root 415M Jul  6 18:31 prep_part2.csv
-rw-r--r-- 1 root root  48M Jul 20 19:59 prep_part2_frontal_face.csv
-rw-r--r-- 1 root root 222M Jul 21 00:33 prep_part2_mid.csv
-rw-r--r-- 1 root root 464M Jul  6 18:38 prep_part3.csv
-rw-r--r-- 1 root root  59M Jul 20 19:43 prep_part3_frontal_face.csv
-rw-r--r-- 1 root root 273M Jul 21 00:32 prep_part3_mid.csv
-rw-r--r-- 1 root root 476M Jul  6 18:42 prep_part4.csv
-rw-r--r-- 1 root root  55M Jul 20 19:11 prep_part4_frontal_face.csv
-rw-r--r-- 1 root root 278M Jul 21 00:26 prep_part4_mid.csv
-rw-r--r-- 1 root root 267M Jul  6 19:15 pre

In [114]:
raw_tag_info = pd.read_csv(prep_trainset_frontal_face_path)
raw_tag_info_df = pd.DataFrame(raw_tag_info)

# raw_tag_info_df["emotion"].value_counts().plot.bar()
# raw_tag_info_df["emotion"].value_counts()

## disgust와 contempt는 disgust 하나로 합침
raw_tag_info_df = raw_tag_info_df.replace('contempt', 'disgust')

## balancing sampling
min_emotion_cnt = np.min(raw_tag_info_df["emotion"].value_counts())
sampled_tag_info_df = raw_tag_info_df.groupby("emotion", group_keys=False).apply(lambda df: df.sample(min_emotion_cnt))
sampled_tag_info_df = sampled_tag_info_df.reset_index(drop=True)

# sampled_tag_info_df["emotion"].value_counts()

# sampled_tag_info_df.count()

# sampled_tag_info_df.head()

### Real-time Convolutional Neural Networks for Emotion and Gender Classification 모델을 위한 전처리 부분
import cv2
import re
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import json
import multiprocessing

# num_cores = multiprocessing.cpu_count()
num_cores = 5

def img_path_to_pixel(x):
    
    img_path, face_rect = x[0], x[1]
    face_rect = json.loads(face_rect.replace("'", "\""))
    
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_shape = img.shape
    
    # face_rect 부분 잘라내기
    # ex) 'max_x': 1125, 'max_y': 798, 'min_x': 708, 'min_y': 267
    min_x = face_rect['min_x']
    max_x = face_rect['max_x']
    min_y = face_rect['min_y']
    max_y = face_rect['max_y']
    
    if  not isinstance(min_x, int) or not isinstance(max_x, int) or  not isinstance(min_y, int) or not isinstance(max_y, int) or \
        min_x <= 0 or max_x <= 0 or min_y <= 0 or max_y <= 0:
        return None
    
#     print(min_y,max_y, min_x,max_x)
    img = img[min_y:max_y, min_x:max_x]
    
    # 이미지 축소 시에는 INTER_AREA, https://076923.github.io/posts/Python-opencv-8/
    img = cv2.resize(img, dsize=(48, 48), interpolation=cv2.INTER_AREA)
    
    # split array to ' ' str
    img = ' '.join(img.astype(int).astype(str).reshape(-1))
    
    return img

def multi_process_sub(df_split):

    return df_split[['img_path', 'face_rect']].apply(img_path_to_pixel, axis=1)

def multi_process(df, func):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# %time sampled_tag_info_df['img_path_new'] = sampled_tag_info_df[['img_path','face_rect']].apply(img_path_to_pixel, axis=1)

%time sampled_tag_info_df['img_path_new'] = multi_process(sampled_tag_info_df, multi_process_sub)

# sampled_tag_info_df.shape

# 이상 데이터 drop
sampled_tag_info_df = sampled_tag_info_df.dropna().reset_index(drop=True)

# sampled_tag_info_df.shape

sampled_tag_info_df['img_path'] = sampled_tag_info_df['img_path_new']
prep_tag_info_df = sampled_tag_info_df.drop(['img_path_new', 'face_rect'], axis=1)

# prep_tag_info_df["emotion"].value_counts().plot.bar()

## 전처리 step 2

# * CSV dataset file 생성
#     * 포멧: emotion
#     * emotion mapping
#         * ASIS: 8 Catogory ('happiness', 'afraid', 'neutral', 'surprise', 'sadness', 'contempt', 'anger', 'disgust')
#         * TO: 7 Category  (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)
#         * mapped Category: 0=anger, 1=disgust&contempt, 2=afraid, 3=happiness, 4=sadness, 5=surprise, 6=neutral

def emotion_mapping(emotion):
    if emotion == 'anger':
        return 0
    elif emotion == 'disgust' or emotion == 'contempt':
        return 1
    elif emotion == 'afraid':
        return 2
    elif emotion == 'happiness':
        return 3
    elif emotion == 'sadness':
        return 4
    elif emotion == 'surprise':
        return 5
    elif emotion == 'neutral':
        return 6
    else:
        return 6
        

prep_tag_info_df['mapped_emotion'] = prep_tag_info_df['emotion'].map(emotion_mapping)

# prep_tag_info_df['mapped_emotion'].value_counts()

prep_tag_info_df['emotion'] = prep_tag_info_df['mapped_emotion']
prep_tag_info_df = prep_tag_info_df.drop(['mapped_emotion'], axis=1)
# prep_tag_info_df.head()

# !ls -alh ../datasets/kor_multi_modal/

prep_tag_info_df.to_csv(prep_trainset_mid_path, index=False)

tmp_df = pd.read_csv(prep_trainset_mid_path)

tmp_df.head()

CPU times: user 282 ms, sys: 624 ms, total: 906 ms
Wall time: 26.1 s


,emotion,img_path
0,2,99 96 102 94 94 92 96 101 100 96 91 95 77 70 8...
1,2,13 15 15 15 17 16 17 16 14 14 14 15 17 20 22 2...
2,2,41 37 37 38 45 47 41 39 46 35 31 34 36 41 43 3...
3,2,20 22 19 19 19 19 20 20 20 20 21 27 30 34 35 3...
4,2,15 15 15 16 16 16 17 19 20 21 22 23 24 23 23 2...


## 파일 병합

In [124]:
# prep_trainset_mid_path = '../datasets/kor_multi_modal/prep_part%d_mid.csv' % 2
# prep_trainset_mid_path
prep_part1_mid = pd.read_csv('../datasets/kor_multi_modal/prep_part%d_mid.csv' % 1)
prep_part2_mid = pd.read_csv('../datasets/kor_multi_modal/prep_part%d_mid.csv' % 2)
prep_part3_mid = pd.read_csv('../datasets/kor_multi_modal/prep_part%d_mid.csv' % 3)
prep_part4_mid = pd.read_csv('../datasets/kor_multi_modal/prep_part%d_mid.csv' % 4)
prep_part5_mid = pd.read_csv('../datasets/kor_multi_modal/prep_part%d_mid.csv' % 5)

train_df = pd.concat([prep_part1_mid, prep_part2_mid, prep_part3_mid, prep_part4_mid, prep_part5_mid])

In [125]:
train_df.shape

(128499, 2)

In [129]:
# column setting to ['emotion', 'pixels', 'Usage']
def column_change(mdf):
    mdf['pixels']=mdf['img_path']
    mdf['Usage']=''
    return mdf.drop(columns=['img_path'])

train_df = column_change(train_df)

In [131]:
train_df.head()

,emotion,pixels,Usage
0,2,99 96 102 94 94 92 96 101 100 96 91 95 77 70 8...,
1,2,13 15 15 15 17 16 17 16 14 14 14 15 17 20 22 2...,
2,2,41 37 37 38 45 47 41 39 46 35 31 34 36 41 43 3...,
3,2,20 22 19 19 19 19 20 20 20 20 21 27 30 34 35 3...,
4,2,15 15 15 16 16 16 17 19 20 21 22 23 24 23 23 2...,


In [133]:
train_df.to_csv('../datasets/kor_multi_modal/prep_col_all_mdf_frontal_face.csv', index=False)